In [11]:
from finassist.utils.database import get_user_context_info

In [ ]:
from google.cloud import bigquery
from finassist.utils.utils import get_var_env

bq_client = None  # <-- Añade esta línea
bq_dataset = None  # <-- Y esta si usas get_bq_dataset

def get_bq_client():
    """Get BigQuery client."""
    global bq_client
    if bq_client is None:
        bq_client = bigquery.Client(project=get_var_env("BQ_PROJECT_ID"))
    return bq_client

def get_bq_dataset():
    """Get BigQuery dataset."""
    global bq_dataset
    if bq_dataset is None:
        bq_dataset = get_bq_client().dataset(get_var_env("BQ_DATASET_ID"))
    return bq_dataset

def get_user_context_info(user_id: str):
    """
    Get user context information from BigQuery.

    Returns:
        dict with user info and account list.
    """
    client = get_bq_client()
    dataset_id = get_var_env("BQ_DATASET_ID")
    project_id = get_var_env("BQ_PROJECT_ID")

    # Construct full table names
    users_table = f"`{project_id}.{dataset_id}.users`"
    accounts_table = f"`{project_id}.{dataset_id}.accounts`"

    # SQL query to get user info and their accounts
    query = f"""
    SELECT
        u.user_id,
        u.first_name,
        u.last_name,
        u.preferred_currency,
        u.language,
        u.timezone,
        ARRAY_AGG(STRUCT(
            a.account_id,
            a.account_name,
            a.account_type,
            a.institution,
            a.currency,
            a.balance
        )) AS accounts
    FROM {users_table} u
    LEFT JOIN {accounts_table} a ON u.user_id = a.user_id
    WHERE u.user_id = @user_id
    GROUP BY u.user_id, u.first_name, u.last_name, u.preferred_currency, u.language, u.timezone
    """

    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("user_id", "STRING", user_id)
        ]
    )

    query_job = client.query(query, job_config=job_config)
    result = list(query_job.result())

    if not result:
        return {
            "error": "No accounts found for the specified user. Ask the user to create an account first."
        }

    row = result[0]

    return {
        "user_id": row.user_id,
        "first_name": row.first_name,
        "last_name": row.last_name,
        "preferred_currency": row.preferred_currency,
        "language": row.language,
        "timezone": row.timezone,
        "accounts": [
            {
                "account_id": acc.account_id,
                "account_name": acc.account_name,
                "account_type": acc.account_type,
                "institution": acc.institution,
                "currency": acc.currency,
                "balance": float(acc.balance) if acc.balance is not None else None,
            }
            for acc in row.accounts if acc.account_id is not None
        ]
    }


NameError: name 'bq_client' is not defined